In [1]:
## TODO 

# Compute CFR for Leduc
# Compute EV for hands in Leduc
# Write update equations for tournament
# Build agents for: multi-arm-bandit,f bayesian update, bayesian optimization, range-update model, DQN?

In [2]:
# UCB MAB baseline
# UCB = empirical mean of specific arm + sqrt(2 log t/N_t), where N_t = number of trials of specific arm
import time, sys
from os.path import dirname, abspath
sys.path.append("/home/leduc/Deep-CFR/")
sys.path.append("/home/leduc/PokerRL/")
import numpy as np
from DeepCFR.EvalAgentDeepCFR import EvalAgentDeepCFR
from PokerRL.game.AgentTournament import AgentTournament
from H2HEvaluator import H2HEval

In [4]:
class MAB:    
    
    def __init__(self, agent_list, test_agent, gamma = 1, n_hands = 2):
        self.agent_list = agent_list
        self.num_agents = len(self.agent_list)
        self.arm_times = [0 for _ in range(self.num_agents)]
        self.ucb = [0 for _ in range(self.num_agents)]
        self.rewards = [0 for _ in range(self.num_agents)]
        self.test_agent = test_agent
        self.gamma = gamma
        self.n_hands = n_hands 
        self.reward_list = [0]
        
    def run(self, n_episodes):
        nash_rewards = []
        avg_nash_rewards = []
        variances = [0]
        nash_agent_path = "/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/120/eval_agentAVRG_NET.pkl"
        nash_agent = EvalAgentDeepCFR.load_from_disk(path_to_eval_agent=nash_agent_path)
        time = np.sum(self.arm_times)
        
        def ucb_updater(time, arm_time):
            if (arm_time == 0):
                return np.infty
            else:
                return np.sqrt(2 * np.log(time) / np.float(arm_time))
            
        for _ in range(n_episodes):
            time += 1
            
            for i in range(self.num_agents):
                self.ucb[i] = self.rewards[i] + self.gamma * ucb_updater(time, self.arm_times[i])
            best_agent_idx = np.argmax(self.ucb)
            
            reward, variance = H2HEval(self.agent_list[best_agent_idx], self.test_agent).h2h_eval(self.n_hands)
            
            print("UCB List: ", self.ucb)
            self.arm_times[best_agent_idx] += 1
            self.rewards[best_agent_idx] = (self.rewards[best_agent_idx] * (self.arm_times[best_agent_idx] - 1) + reward) / self.arm_times[best_agent_idx]
            print("best agent, reward, arm_times, avg reward", best_agent_idx, reward,self.arm_times[best_agent_idx], self.rewards[best_agent_idx])
            self.reward_list.append((self.reward_list[-1] * (time - 1) + reward) / time)
            
            nash_rewards.append(H2HEval(nash_agent, self.test_agent).h2h_eval(self.n_hands)[0])
            avg_nash_rewards.append(np.mean(nash_rewards))
            variances.append((variances[-1] * (time - 1) ** 2 + variance) / time ** 2)
            
        lower_bounds = [self.reward_list[i] - 1.96 * np.sqrt(variances[i]) for i in range(n_episodes)]
        upper_bounds = [self.reward_list[i] + 1.96 * np.sqrt(variances[i]) for i in range(n_episodes)]
        plt.plot(self.reward_list)
        plt.fill_between(range(n_episodes), upper_bounds, lower_bounds, color='blue', alpha=.5)
        plt.plot(avg_nash_rewards, color = 'red')
        plt.title('Performance of Multiarmed Bandit (Blue) vs. Nash (Red)')
        plt.ylabel('Average Reward Per Hand')
        plt.xlabel('Time')
        plt.show()
        
        print("number of trials of each arm:", self.arm_times)

In [5]:
# 0 = fold
# 1 = call
# 2 = raise

In [4]:
from PokerRL.game.games import StandardLeduc  # or any other game

from DeepCFR.EvalAgentDeepCFR import EvalAgentDeepCFR
from DeepCFR.TrainingProfile import TrainingProfile
from DeepCFR.workers.driver.Driver import Driver

t_prof = TrainingProfile(name="MAB",
                                         nn_type="feedforward",
                                         max_buffer_size_adv=3e6,
                                         eval_agent_export_freq=20,  # export API to play against the agent
                                         n_traversals_per_iter=1500,
                                         n_batches_adv_training=750,
                                         n_batches_avrg_training=2000,
                                         n_merge_and_table_layer_units_adv=64,
                                         n_merge_and_table_layer_units_avrg=64,
                                         n_units_final_adv=64,
                                         n_units_final_avrg=64,
                                         mini_batch_size_adv=2048,
                                         mini_batch_size_avrg=2048,
                                         init_adv_model="last",
                                         init_avrg_model="last",
                                         use_pre_layers_adv=False,
                                         use_pre_layers_avrg=False,

                                         game_cls=StandardLeduc,

                                         # You can specify one or both modes. Choosing both is useful to compare them.
                                         eval_modes_of_algo=(
                                             EvalAgentDeepCFR.EVAL_MODE_SINGLE,  # SD-CFR
                                             EvalAgentDeepCFR.EVAL_MODE_AVRG_NET,  # Deep CFR
                                         ),

                                         DISTRIBUTED=False,
                                         )
fold_agent = FoldAgent(t_prof=t_prof)
bet_agent = BetAgent(t_prof=t_prof)

ModuleNotFoundError: No module named 'PokerRL'

In [8]:
avg_agents = ["/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/" + str(20 * i) + "/eval_agentAVRG_NET.pkl" for i in range(1,6)]
single_agents = ["/home/leduc/poker_žai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/" + str(20 * i) + "/eval_agentSINGLE.pkl" for i in range(1,6)]
agents = avg_agents + single_agents
print(agents, "bet agent")
agents = [EvalAgentDeepCFR.load_from_disk(path_to_eval_agent=agents[i]) for i in range(len(agents))]
agents.append(bet_agent)

['/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/20/eval_agentAVRG_NET.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/40/eval_agentAVRG_NET.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/60/eval_agentAVRG_NET.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/80/eval_agentAVRG_NET.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/100/eval_agentAVRG_NET.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/20/eval_agentSINGLE.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/40/eval_agentSINGLE.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/60/eval_agentSINGLE.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/80/eval_agentSINGLE.pkl', '/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_200/100/eval_agentSINGLE.pkl'] bet agent


In [1]:
agents = agents[:-1]
random_agent_path = "/home/leduc/poker_ai_data/eval_agent/SD-CFR_LEDUC_EXAMPLE_2/2/eval_agentAVRG_NET.pkl"
random_agent = EvalAgentDeepCFR.load_from_disk(path_to_eval_agent=random_agent_path)

NameError: name 'agents' is not defined

In [ ]:
bandit = MAB(agents, random_agent, n_hands = 50)
bandit.run(500)

In [ ]:
bandit = MAB(agents, random_agent, gamma = 0.5, n_hands = 10)
bandit.run(500)

In [ ]:
bandit = MAB(agents, random_agent, gamma = 10, n_hands = 10)
bandit.run(500)